In [4]:
from osmnx import settings
import osmnx as ox
import matplotlib.pyplot as plt
from IPython.display import display, Image
from ipywidgets import Layout, Text, SelectMultiple, ColorPicker, Button, Output, HTML

settings.use_cache = True



# Function to create and display the road network map
def create_map(city_name, network_types, road_color, bg_color, text_color):
    try:
        
        city_polygon = ox.geocode_to_gdf(city_name).geometry.to_list()[0]
        formatted_network_types = "|".join(network_types)
        network_type_filter = f'["highway"~"{formatted_network_types}"]'
        graph = ox.graph_from_polygon(city_polygon, custom_filter=network_type_filter)
        nodes, edges = ox.graph_to_gdfs(graph)

        fig, ax = plt.subplots(figsize=(10, 10))
        ax.set_aspect('equal')

        road_color = road_color if road_color else '#ffffff'
        bg_color = bg_color if bg_color else '#000000'
        edges.plot(ax=ax, color=road_color)
        ax.set_facecolor(bg_color)

        ax.set_xticks([])
        ax.set_yticks([])

        y_range = ax.get_ylim()
        ax.set_ylim(y_range[0], y_range[1] + 0.2 * (y_range[1] - y_range[0]))
        ax.set_title(city_name, fontsize=30, color=text_color, y=0.9)
        
        plt.show()

    except Exception as e:
        print(f"An error occurred: {e}")

# Widgets for user inputs
city_input = Text(
    value='',
    placeholder='City, State (if any), Country in full',
    description='City:',
    disabled=False,
    layout = Layout(width='50%'),
    tooltip='Enter the city name',
    _dom_classes=["custom-description"]

)

network_types_input = SelectMultiple(
    options=["motorway","motorway_link", "trunk","trunk_link", "primary","primary_link", "secondary","secondary_link", "tertiary","tertiary_link" ],
    value=["motorway","motorway_link", "trunk","trunk_link", "primary","primary_link", "secondary", "tertiary"],
    description='Network Types',
    layout = Layout(width='50%'),
    disabled=False,
    _dom_classes=["custom-description"])

road_color_input = ColorPicker(
    value='white',
    description='Road Color',
    disabled=False,
    layout = Layout(width='50%'),
    _dom_classes=["custom-description"]
)

bg_color_input = ColorPicker(
    value='black',
    description='BG Color',
    disabled=False,
    layout = Layout(width='50%'),
    _dom_classes=["custom-description"])

text_color_input = ColorPicker(
    value='red', 
    description='Text Color',
    disabled=False,
    layout = Layout(width='50%'),
    _dom_classes=["custom-description"])

submit_button = Button(
    description='Submit',
    disabled=False,
    layout = Layout(width='50%'),
    button_style='',
    tooltip='Submit',
    icon='check',
    _dom_classes=["custom-description"])

output = Output()

def on_submit_button_clicked(b):
    with output:
        output.clear_output()
        create_map(city_input.value, network_types_input.value, road_color_input.value, bg_color_input.value, text_color_input.value)

submit_button.on_click(on_submit_button_clicked)

# Display the widgets and output
display(HTML("""
<style>
.custom-description label{
    font-size: 15px;
    color: red !important;
}
</style>
"""))

display(city_input, network_types_input, road_color_input, bg_color_input,text_color_input, submit_button, output)

HTML(value='\n<style>\n.custom-description label{\n    font-size: 15px;\n    description_width: initial;\n    …

Text(value='', description='City:', layout=Layout(width='50%'), placeholder='City, State (if any), Country in …

SelectMultiple(_dom_classes=('custom-description',), description='Network Types', index=(0, 1, 2, 3, 4, 5, 6, …

ColorPicker(value='white', description='Road Color', layout=Layout(width='50%'), _dom_classes=('custom-descrip…

ColorPicker(value='black', description='BG Color', layout=Layout(width='50%'), _dom_classes=('custom-descripti…

ColorPicker(value='red', description='Text Color', layout=Layout(width='50%'), _dom_classes=('custom-descripti…

Button(description='Submit', icon='check', layout=Layout(width='50%'), style=ButtonStyle(), tooltip='Submit', …

Output()